# Beginning

Dans le cadre de notre formation en data science, nous menons un projet d’analyse visant à prédire le risque de défaut de crédit. Les institutions financières doivent en effet évaluer ce risque afin de réduire les pertes et d’optimiser leurs décisions. En mobilisant des techniques de machine learning, notre objectif est de construire et comparer des modèles prédictifs capables d’améliorer l’évaluation du risque et de proposer des recommandations utiles pour le processus d’octroi de crédit.

# Overview

Dans ce projet, nous suivrons une démarche progressive en plusieurs étapes. Nous commencerons par une phase d’exploration des données, consistant à analyser les variables disponibles, détecter les valeurs manquantes et identifier les relations clés entre caractéristiques et risque de défaut. Cette étape sera suivie d’un prétraitement des données, incluant le nettoyage, la transformation et éventuellement l’ingénierie de nouvelles variables afin d’optimiser la qualité des entrées du modèle. Ensuite, nous aborderons la modélisation, où différents algorithmes de machine learning (régression logistique, arbres de décision, forêts aléatoires, gradient boosting, etc.) seront construits et entraînés. Une phase d’évaluation permettra de comparer ces modèles à travers des métriques adaptées (accuracy, précision, rappel, F1-score, AUC-ROC) et d’interpréter leurs performances. Sur cette base, nous formulerons des recommandations pratiques quant au choix du modèle le plus pertinent et à son utilisation dans le processus décisionnel des institutions financières. Enfin, nous proposerons des pistes de perfectionnement et des étapes futures, telles que l’intégration de nouvelles sources de données ou le recours à des méthodes plus avancées comme le deep learning.

# Business Understanding

L’octroi de crédit représente un enjeu majeur pour les institutions financières, car un défaut de remboursement peut générer des pertes importantes. Les méthodes traditionnelles d’évaluation du risque, souvent basées sur des modèles statistiques simples, ne captent pas toujours la complexité des comportements des emprunteurs. Dans ce projet, nous utilisons le machine learning pour prédire la probabilité de défaut en analysant les caractéristiques des clients, ce qui permet d’améliorer la précision des décisions de crédit et de mieux gérer les risques financiers.

# Data Understanding
Pour ce projet de prédiction de défaut de crédit, nous avons sélectionné le dataset "Give Me Some Credit" issu de Kaggle, qui contient 150 000 observations et 11 variables, garantissant des données réalistes et pertinentes pour notre objectif métier : prédire si un emprunteur connaîtra des difficultés financières sérieuses dans les deux années suivantes (variable cible SeriousDlqin2yrs), 1 indique que l’emprunteur a connu des difficultés financières sérieuses (défaut ou retard de paiement important) dans les deux années suivantes, et 0 indique l’absence de tels événements. Bien que Kaggle fournisse un split pré-établi (training et test), nous avons choisi d’utiliser uniquement le fichier training (cs-training.csv) contenant les labels, afin de contrôler entièrement notre stratégie de validation. Nous créerons notre propre division train/test avec stratification pour maintenir la proportion des classes et évaluer nos modèles de manière rigoureuse avec des métriques fiables, le fichier test original ne permettant pas cette évaluation dans le cadre académique.

Les variables présentes dans ce dataset couvrent les dimensions essentielles de l'évaluation du risque crédit telles qu'utilisées par les institutions financières. D'une part, nous disposons d'indicateurs financiers directs comme le ratio d'endettement (DebtRatio), les revenus mensuels (MonthlyIncome), et l'utilisation des lignes de crédit (RevolvingUtilizationOfUnsecuredLines), qui reflètent la situation économique actuelle de l'emprunteur. D'autre part, des variables comportementales historiques telles que les retards de paiement sur différentes périodes (NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate) permettent d'évaluer la discipline de paiement passée, considérée comme un prédicteur fort du comportement futur. Les variables démographiques et contextuelles complètent ce profil de risque en apportant des informations sur l'âge, le nombre de personnes à charge (NumberOfDependents), et le portefeuille de crédits existants (NumberOfOpenCreditLinesAndLoans, NumberRealEstateLoansOrLines).

# Data Preparation

In [15]:
# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [16]:
# Chargement des données
df = pd.read_csv('credit.csv', index_col=0)

print(f"Dataset chargé: {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
print(f"Colonnes: {list(df.columns)}")

# Informations générales avant nettoyage
print("\nTypes de données:")
print(df.dtypes)

print(f"\nStatistiques générales:")
print(f"- Nombre total de lignes: {df.shape[0]:,}")
print(f"- Nombre de colonnes: {df.shape[1]}")

# Suppression des lignes avec valeurs manquantes
df = df.dropna()


# Informations générales après nettoyage
print(f"\nNouveau shape: {df.shape[0]:,} lignes × {df.shape[1]} colonnes")

df.head()

Dataset chargé: 150,000 lignes × 11 colonnes
Colonnes: ['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

Types de données:
SeriousDlqin2yrs                          int64
RevolvingUtilizationOfUnsecuredLines    float64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
MonthlyIncome                           float64
NumberOfOpenCreditLinesAndLoans           int64
NumberOfTimes90DaysLate                   int64
NumberRealEstateLoansOrLines              int64
NumberOfTime60-89DaysPastDueNotWorse      int64
NumberOfDependents                      float64
dtype: object

Statistiques générales:
- Nombre total de lignes: 150,000
- Nombre de colonnes: 11

Nouve

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [17]:
# Préparation des données pour la modélisation

# Séparation X et y
X = df.drop('SeriousDlqin2yrs', axis=1)
y = df['SeriousDlqin2yrs']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Division train/test avec stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape[0]:,} observations")
print(f"Test set: {X_test.shape[0]:,} observations")

# Vérification de la distribution
print(f"\nDistribution train: {y_train.value_counts(normalize=True).sort_index().values}")
print(f"Distribution test: {y_test.value_counts(normalize=True).sort_index().values}")

# Scaling des données (fit sur train seulement)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Données préparées et scalées")
print(f"X_train_scaled: {X_train_scaled.shape}")
print(f"X_test_scaled: {X_test_scaled.shape}")

Features (X): (120269, 10)
Target (y): (120269,)
Training set: 96,215 observations
Test set: 24,054 observations

Distribution train: [0.9305098 0.0694902]
Distribution test: [0.9305313 0.0694687]

✅ Données préparées et scalées
X_train_scaled: (96215, 10)
X_test_scaled: (24054, 10)


# Modeling

In [18]:
# -------------------------------
# Régression Logistique - Interprétation avant Evaluation
# -------------------------------


#  Entraînement du modèle
logreg = LogisticRegression(class_weight='balanced', random_state=42)
logreg.fit(X_train_scaled, y_train)

#  Coefficients et intercept
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': logreg.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print("### Coefficients du modèle ###")
display(coef_df)
print("Intercept:", logreg.intercept_[0])

#  Probabilités prédites pour quelques observations
probas_df = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted_Probability': logreg.predict_proba(X_test_scaled)[:10,1]
})
print("\n### Probabilités prédites pour les 10 premières observations ###")
display(probas_df)



from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#  Arbre de Décision simple
tree = DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42)
tree.fit(X_train, y_train)

# Prédictions et probabilités
y_pred_tree = tree.predict(X_test)
y_proba_tree = tree.predict_proba(X_test)[:,1]


# Affichage des probabilités pour les 10 premières observations
probas_tree_df = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted_Probability': y_proba_tree[:10]
})
print("### Probabilités prédites - Arbre de Décision ###")
display(probas_tree_df)



#  Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=7, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

# Prédictions et probabilités
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

# Affichage des probabilités pour les 10 premières observations
probas_rf_df = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted_Probability': y_proba_rf[:10]
})
print("### Probabilités prédites - Random Forest ###")
display(probas_rf_df)



### Coefficients du modèle ###


,Feature,Coefficient
6,NumberOfTimes90DaysLate,2.181607
2,NumberOfTime30-59DaysPastDueNotWorse,1.975843
8,NumberOfTime60-89DaysPastDueNotWorse,0.707231
7,NumberRealEstateLoansOrLines,0.110565
9,NumberOfDependents,0.075989
5,NumberOfOpenCreditLinesAndLoans,0.026577
0,RevolvingUtilizationOfUnsecuredLines,-0.007177
3,DebtRatio,-0.071602
1,age,-0.397077
4,MonthlyIncome,-0.468878


Intercept: -0.21934069927367913

### Probabilités prédites pour les 10 premières observations ###


,Actual,Predicted_Probability
0,0,0.299329
1,0,0.275603
2,0,0.375185
3,0,0.961176
4,0,0.496856
5,0,0.325039
6,0,0.262725
7,0,0.815411
8,0,0.406870
9,0,0.324655


### Probabilités prédites - Arbre de Décision ###


,Actual,Predicted_Probability
0,0,0.539453
1,0,0.349023
2,0,0.290031
3,0,0.835614
4,0,0.138925
5,0,0.138925
6,0,0.138925
7,0,0.921013
8,0,0.290031
9,0,0.138925


### Probabilités prédites - Random Forest ###


,Actual,Predicted_Probability
0,0,0.414792
1,0,0.421280
2,0,0.269036
3,0,0.781522
4,0,0.151019
5,0,0.233385
6,0,0.105626
7,0,0.902777
8,0,0.191295
9,0,0.143732


L’analyse des coefficients de la régression logistique met en évidence que les variables liées aux retards de paiement (NumberOfTimes90DaysLate, NumberOfTime30-59DaysPastDueNotWorse, NumberOfTime60-89DaysPastDueNotWorse) sont les facteurs les plus déterminants dans l’évaluation du risque de non-remboursement, avec des coefficients fortement positifs indiquant qu’une augmentation de ces variables accroît significativement la probabilité qu’un client connaisse des difficultés financières. À l’inverse, l’âge et le revenu mensuel jouent un rôle protecteur en réduisant ce risque. Les résultats pour quelques observations montrent que le modèle identifie correctement certains profils à haut risque (par exemple une estimation proche de 0.96 pour un cas réel d’emprunteur fragile). L’arbre de décision et la Random Forest confirment cette tendance en attribuant également des scores de risque plus élevés aux individus présentant des signaux de fragilité financière, tout en offrant une meilleure capacité de capture des relations complexes entre variables. Concrètement, cela signifie que notre approche de modélisation permet non seulement de détecter les emprunteurs les plus susceptibles de rencontrer des difficultés de remboursement, mais aussi d’apporter une explication claire sur les variables clés que les institutions financières devraient surveiller dans leur processus d’octroi de crédit.

# Evaluation



In [19]:
# Évaluation des modèles
models = {'Logistic Regression': (y_pred_logreg, y_proba_logreg),
          'Decision Tree': (y_pred_tree, y_proba_tree),
          'Random Forest': (y_pred_rf, y_proba_rf)}

for name, (y_pred, y_proba) in models.items():
    print(f"\n{name} Performance:")
    print(f"Précision   : {precision_score(y_test, y_pred):.3f}")
    print(f"Rappel      : {recall_score(y_test, y_pred):.3f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, y_proba):.3f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.3f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


Logistic Regression Performance:
Précision   : 0.201
Rappel      : 0.618
AUC-ROC: 0.783
F1-Score: 0.304
Confusion Matrix:
[[18291  4092]
 [  639  1032]]

Decision Tree Performance:
Précision   : 0.182
Rappel      : 0.778
AUC-ROC: 0.829
F1-Score: 0.296
Confusion Matrix:
[[16559  5824]
 [  371  1300]]

Random Forest Performance:
Précision   : 0.212
Rappel      : 0.743
AUC-ROC: 0.846
F1-Score: 0.330
Confusion Matrix:
[[17784  4599]
 [  430  1241]]


#### Interprétation des résultats des modèles

Les trois modèles présentent des performances intéressantes, mais avec des nuances. La régression logistique obtient un AUC-ROC de 0,783, ce qui traduit une capacité correcte à distinguer les bons et les mauvais payeurs, mais sa précision est faible (0,201), indiquant que beaucoup de clients signalés à risque ne le sont pas réellement. Son rappel (0,618) montre néanmoins qu’elle identifie une proportion correcte des mauvais payeurs. L’arbre de décision améliore le rappel (0,778), captant davantage de clients réellement à risque, mais sa précision (0,182) reste faible et il génère plus de faux positifs, ce qui peut créer une surcharge pour l’entreprise si elle agit préventivement sur chaque cas. Enfin, la forêt aléatoire équilibre mieux les métriques : elle obtient le meilleur AUC-ROC (0,846), une précision légèrement plus élevée (0,212) et un rappel robuste (0,743), ce qui lui permet à la fois de détecter une majorité des clients à risque tout en réduisant les fausses alertes par rapport à l’arbre de décision.

#### Choix du meilleur modèle

Parmi les trois approches, la forêt aléatoire ressort comme la meilleure option pour le projet. Elle combine une capacité de discrimination supérieure (AUC-ROC le plus élevé), un rappel élevé (indispensable pour limiter les pertes liées aux défauts de paiement) et une précision relativement meilleure, réduisant ainsi le coût opérationnel lié aux faux positifs. Ce compromis est particulièrement adapté au contexte de gestion du risque de crédit, où il vaut mieux prévenir le maximum de défauts tout en limitant les interventions inutiles. En outre, la robustesse et la nature ensemble de la forêt aléatoire la rendent plus fiable que le simple arbre de décision, tout en offrant une flexibilité supérieure à la régression logistique face aux relations complexes entre variables.

# Recommendations


La régression logistique identifie que certains clients présentent un risque élevé de non-remboursement. Par exemple :

Le rappel de 0,618 montre que le modèle repère environ 62 % des clients qui connaîtront effectivement des difficultés de paiement.

La précision de 0,201 indique que parmi les clients signalés à risque, seulement 20 % connaîtront réellement des problèmes, ce qui signifie que beaucoup de clients sont préventivement signalés mais ne poseront pas de problème.

La matrice de confusion révèle que sur 1 671 clients réellement à risque (639+1032), 1 032 sont correctement identifiés, mais 639 échappent à la détection.

En pratique pour la banque :

Les clients identifiés comme à risque peuvent faire l’objet d’un suivi accru, d’un ajustement du montant du crédit ou de garanties supplémentaires.

Cependant, la banque doit être consciente qu’un grand nombre de clients signalés à risque ne le sont pas réellement, et donc un filtrage complémentaire ou un seuil plus strict pourrait être envisagé pour éviter des interventions inutiles.

# Next Steps

Il est important de noter que la faible précision et le F1-score modéré des modèles ne traduisent pas forcément une mauvaise performance des algorithmes, mais reflètent plutôt la nature déséquilibrée et limitée du dataset. La proportion relativement faible de clients en défaut et l’absence de certaines variables explicatives rendent difficile la distinction parfaite entre clients à risque et clients sûrs, ce qui explique les erreurs de classification observées malgré un AUC satisfaisant.

# Thank you
Ce projet a permis d’explorer l’utilisation de techniques de machine learning pour la prédiction du risque de défaut de crédit à partir du dataset “Give Me Some Credit”. Nous avons comparé plusieurs approches : régression logistique, arbre de décision et forêt aléatoire. Les résultats montrent que la forêt aléatoire offre la meilleure performance globale, en combinant un bon rappel, une précision supérieure aux autres modèles et le meilleur AUC-ROC, ce qui en fait l’outil le plus adapté pour identifier les clients à risque tout en limitant les interventions inutiles.

L’analyse des résultats met également en évidence l’importance des variables liées aux retards de paiement et aux informations financières personnelles dans la prédiction du risque. Enfin, il est important de noter que la performance limitée en précision et F1-score est en partie liée à la nature déséquilibrée du dataset, ce qui souligne la nécessité d’une évaluation prudente et d’une amélioration continue du modèle.

En pratique, l’approche développée permettrait à une institution financière de mieux cibler ses décisions de crédit, d’appliquer des garanties adaptées pour les clients à risque et d’optimiser ses processus de prêt tout en réduisant les pertes financières.
